In [0]:
%sql
MERGE INTO projetos.gold.ecommerce_sales_agg AS target
USING (                              
  SELECT event_date, 
        event_timestamp, 
        customer_id,
        invoice_no, 
        SUM(quantity) as items_carts, 
        CAST(SUM(total_price) AS DECIMAL(18,2)) as cart_price, 
        country
  FROM (
    SELECT 
      event_date, 
      event_timestamp, 
      customer_id,
      invoice_no, 
      quantity, 
      (quantity * unit_price) as total_price, 
      country
    FROM projetos.silver.ecommerce_transactions
  )
  GROUP BY ALL
) AS source
ON  target.event_date      = source.event_date
AND target.event_timestamp = source.event_timestamp
AND target.invoice_no      = source.invoice_no
AND target.customer_id     = source.customer_id
AND target.items_carts     = source.items_carts
AND target.cart_price      = source.cart_price
AND target.country         = source.country
WHEN NOT MATCHED THEN
  INSERT (event_date, event_timestamp, invoice_no, customer_id, items_carts, cart_price, country)
  VALUES (source.event_date, source.event_timestamp, source.invoice_no, source.customer_id, source.items_carts, source.cart_price, source.country)

In [0]:
%sql
OPTIMIZE projetos.gold.ecommerce_sales_agg;

In [0]:
%sql
ALTER TABLE projetos.gold.ecommerce_sales_agg SET TAGS (
  'last_run' = '${runtime}'
)